In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import csv
import pickle

In [2]:
ratings = pd.read_csv('../../Dataset/ml-latest-small/ratings.csv')
movies = pd.read_csv('../../Dataset/ml-latest-small/movies.csv')


#we are going to extract first 5 users for debugging causes
#the first 561 rows are the first 5 users
#code must be written flexible to change the data into the 25m dataset
#first we have to merge all the data movieid, userid, rating, year, genres, etc. so everything is all together

#goal is to:
#1- sort the data descending according to rating counts by their genres 
#2- divide each genre's score by genretotalcount and get a float number
#3- reverse all the float number scores and this score is each genre's score
#5- say if movie has 2 genres, drama and romance. sum romance and drama score and multiply by total count. this is how much 
#this movie should be repeated

#we need three files:
#1- item data: movieid | year | average rating(movie's) | 14 genres
#2- user data: userid | rating count | rating average(user's) | 14 genres
#3- true ratings of movies (y)

#to get these files it is easier to work with a dataset that has all the columns


In [3]:
#first we have to find how much rating each genre has
#movies = movieid | title("movie name" ("movie year")) | genres(seperated by "|") we have to one hot encode genres and seperate the year
#from the title

In [4]:
#checking for how many movies there are
print(ratings['movieId'].nunique())
print(movies['movieId'].nunique())

9724
9742


In [5]:
##dropping first 5 users ratings from ratings dataset for debugging purposes
#first_5_users_df = ratings.head(560)
#ratings = ratings.drop(ratings.index[:560])


In [6]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [7]:
#we have to one hot encode genres
genres = movies['genres'].str.get_dummies('|')
#dropping some columns
genres.drop(columns=['(no genres listed)', 'Film-Noir', 'IMAX', 'Musical', 'Western', 'War'] ,axis=1, inplace=True)
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,1,0,1,0,0,0,1,0,0,0,0,0
9738,0,0,1,0,1,0,0,0,1,0,0,0,0,0
9739,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9740,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [8]:
#we made this dataframe for using to make item_vecs and for movie_analysis dataframe
movies_w_genres = pd.concat([movies, genres], axis=1)

In [9]:
#merging one hot encoded genres with movies dataframe
movies = pd.concat([movies, genres], axis=1)
movies.drop(columns='genres', axis=1, inplace=True)
movies

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),1,0,1,0,1,0,0,0,1,0,0,0,0,0
9738,193583,No Game No Life: Zero (2017),0,0,1,0,1,0,0,0,1,0,0,0,0,0
9739,193585,Flint (2017),0,0,0,0,0,0,0,1,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [10]:
#extarct year from its title
pattern = '\((\d{4})\)'
movies['year'] = movies.title.str.extract(pattern, expand=False)


In [11]:
#finding movieid's has null values
null_ids = movies[movies['year'].isnull()]['movieId']
null_ids = pd.DataFrame(null_ids)
#dataframe  has null movieids
null_ids
null_id_array = null_ids['movieId'].values
#array has null movieids
null_id_array

array([ 40697, 140956, 143410, 147250, 149334, 156605, 162414, 167570,
       171495, 171631, 171749, 171891, 176601], dtype=int64)

In [12]:
#we got the years of the movies manually from internet and we need to merge it with movies dataset
years_of_null_movies = ["1994", "2018", "2015", "1979", "2016", "2016", "2016", "2016", "2019", "2017", "2006", "2017", "2011"]
null_ids['year'] = years_of_null_movies

null_ids

,movieId,year
6059,40697,1994
9031,140956,2018
9091,143410,2015
9138,147250,1979
9179,149334,2016
9259,156605,2016
9367,162414,2016
9448,167570,2016
9514,171495,2019
9515,171631,2017


In [13]:
#filled movies df's year columns with the values that we found manually
movies['year'] = movies['year'].fillna(null_ids['year'])

In [14]:
ratings.drop(columns='timestamp', inplace=True, axis=1)
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [15]:
#gerelerı ratings dfle birleştir, groupbylı bi şeylerle topla
#genre countları bul

In [16]:
#reindex year column
movies = movies.reindex(columns=['movieId', 'title', 'year', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller'])
movies

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),1995,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),1995,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),1995,0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),1995,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),2017,1,0,1,0,1,0,0,0,1,0,0,0,0,0
9738,193583,No Game No Life: Zero (2017),2017,0,0,1,0,1,0,0,0,1,0,0,0,0,0
9739,193585,Flint (2017),2017,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),2018,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [17]:
#merge rating df with their genres
merged_ratings = pd.merge(ratings, movies, on='movieId', how='outer')
merged_ratings

,userId,movieId,rating,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1.0,1,4.0,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,5.0,1,4.0,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
2,7.0,1,4.5,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
3,15.0,1,2.5,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
4,17.0,1,4.5,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100849,NaN,30892,NaN,In the Realms of the Unreal (2004),2004,0,0,1,0,0,0,1,0,0,0,0,0,0,0
100850,NaN,32160,NaN,Twentieth Century (1934),1934,0,0,0,0,1,0,0,0,0,0,0,0,0,0
100851,NaN,32371,NaN,Call Northside 777 (1948),1948,0,0,0,0,0,1,0,1,0,0,0,0,0,0
100852,NaN,34482,NaN,"Browning Version, The (1951)",1951,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [18]:
#drop unrated movies
merged_ratings = merged_ratings.drop(labels=range(100836, 100854), axis=0)
merged_ratings

,userId,movieId,rating,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1.0,1,4.0,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,5.0,1,4.0,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
2,7.0,1,4.5,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
3,15.0,1,2.5,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
4,17.0,1,4.5,Toy Story (1995),1995,0,1,1,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610.0,160341,2.5,Bloodmoon (1997),1997,1,0,0,0,0,0,0,0,0,0,0,0,0,1
100832,610.0,160527,4.5,Sympathy for the Underdog (1971),1971,1,0,0,0,0,1,0,1,0,0,0,0,0,0
100833,610.0,160836,3.0,Hazard (2005),2005,1,0,0,0,0,0,0,1,0,0,0,0,0,1
100834,610.0,163937,3.5,Blair Witch (2016),2016,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [19]:
#find each genre's count
genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']

genres_dict = {}

for i in genres:
    genres_dict[i] = merged_ratings[i].sum()

genres_dict = {k: v for k, v in sorted(genres_dict.items(), key=lambda item: item[1])}
genres_dict

{'Documentary': 1219,
 'Animation': 6988,
 'Horror': 7291,
 'Mystery': 7674,
 'Children': 9208,
 'Fantasy': 11834,
 'Crime': 16681,
 'Sci-Fi': 17243,
 'Romance': 18124,
 'Adventure': 24161,
 'Thriller': 26452,
 'Action': 30635,
 'Comedy': 39053,
 'Drama': 41928}

In [20]:
#count of each genres repeated is 258491 (genretotalcount)
#rating count is 100837
#we calculated the genre scores (genre's count / genres total count)

genre_scores = {}

for i in genres_dict.keys():
    genre_scores[i] = genres_dict[i] / 258491

genre_scores = {k: v for k, v in sorted(genre_scores.items(), key=lambda item: item[1], reverse=True)}
genre_scores


{'Drama': 0.16220293936732808,
 'Comedy': 0.15108069526598605,
 'Action': 0.11851476453725662,
 'Thriller': 0.10233238294563447,
 'Adventure': 0.09346940512435636,
 'Romance': 0.0701146268148601,
 'Sci-Fi': 0.0667063843615445,
 'Crime': 0.06453222742764739,
 'Fantasy': 0.045781091024445726,
 'Children': 0.03562212997744602,
 'Mystery': 0.029687687385634317,
 'Horror': 0.028206011041003363,
 'Animation': 0.027033823227888013,
 'Documentary': 0.0047158314989690165}

In [21]:
#we flipped the score values so we can represent the underrated genres more

sorted_values = sorted(genre_scores.values())

reversed_genre_scores = {k: v for k, v in zip(genre_scores.keys(), sorted_values)}
reversed_genre_scores

{'Drama': 0.0047158314989690165,
 'Comedy': 0.027033823227888013,
 'Action': 0.028206011041003363,
 'Thriller': 0.029687687385634317,
 'Adventure': 0.03562212997744602,
 'Romance': 0.045781091024445726,
 'Sci-Fi': 0.06453222742764739,
 'Crime': 0.0667063843615445,
 'Fantasy': 0.0701146268148601,
 'Children': 0.09346940512435636,
 'Mystery': 0.10233238294563447,
 'Horror': 0.11851476453725662,
 'Animation': 0.15108069526598605,
 'Documentary': 0.16220293936732808}

In [22]:
#we convert dictionary to dataframe

genre_score_df = pd.DataFrame.from_dict(reversed_genre_scores, orient='index')
genre_score_df = genre_score_df.transpose()
genre_score_df

,Drama,Comedy,Action,Thriller,Adventure,Romance,Sci-Fi,Crime,Fantasy,Children,Mystery,Horror,Animation,Documentary
0,0.004716,0.027034,0.028206,0.029688,0.035622,0.045781,0.064532,0.066706,0.070115,0.093469,0.102332,0.118515,0.151081,0.162203


In [23]:
#assign each genres score to the one hot encoded genres df

for i in genre_score_df.columns:
    movies[i].replace(1, genre_score_df[i][0], inplace=True)

movies

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),1995,0.000000,0.035622,0.151081,0.093469,0.027034,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0
1,2,Jumanji (1995),1995,0.000000,0.035622,0.000000,0.093469,0.000000,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0
2,3,Grumpier Old Men (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.0,0.0,0.000000,0.000000,0.0,0.0,0.045781,0.0,0.0
3,4,Waiting to Exhale (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.0,0.0,0.004716,0.000000,0.0,0.0,0.045781,0.0,0.0
4,5,Father of the Bride Part II (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),2017,0.028206,0.000000,0.151081,0.000000,0.027034,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0
9738,193583,No Game No Life: Zero (2017),2017,0.000000,0.000000,0.151081,0.000000,0.027034,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0
9739,193585,Flint (2017),2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.004716,0.000000,0.0,0.0,0.000000,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),2018,0.028206,0.000000,0.151081,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0


In [24]:
#now we have to sum all the scores each movie has
movies['movie_score'] = movies.iloc[:,3:].sum(axis=1)
movies

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,movie_score
0,1,Toy Story (1995),1995,0.000000,0.035622,0.151081,0.093469,0.027034,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0,0.377321
1,2,Jumanji (1995),1995,0.000000,0.035622,0.000000,0.093469,0.000000,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0,0.199206
2,3,Grumpier Old Men (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.0,0.0,0.000000,0.000000,0.0,0.0,0.045781,0.0,0.0,0.072815
3,4,Waiting to Exhale (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.0,0.0,0.004716,0.000000,0.0,0.0,0.045781,0.0,0.0,0.077531
4,5,Father of the Bride Part II (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.027034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),2017,0.028206,0.000000,0.151081,0.000000,0.027034,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0,0.276435
9738,193583,No Game No Life: Zero (2017),2017,0.000000,0.000000,0.151081,0.000000,0.027034,0.0,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.0,0.0,0.248229
9739,193585,Flint (2017),2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.004716,0.000000,0.0,0.0,0.000000,0.0,0.0,0.004716
9740,193587,Bungo Stray Dogs: Dead Apple (2018),2018,0.028206,0.000000,0.151081,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.179287


In [25]:

movies.sort_values(by=['movie_score']).head(85)#81 tane film gidicek bb movie score = 0
#sadece sildiğimiz genrelara sahip olan filmlere ne yapıcaz?
#tekrar sayısı çok yüksek?????
    #genre sayısına bölebiliriz
    #1000' gibi bi şeye bölebiliriz
    #başka bi şeyler?

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,movie_score
700,918,Meet Me in St. Louis (1944),1944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3828,5372,Calamity Jane (1953),1953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8782,129250,Superfast! (2015),2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
178,210,Wild Bill (1995),1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4633,6911,"Jolson Story, The (1946)",1946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,416,Bad Girls (1994),1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4862,7265,"Dreamers, The (2003)",2003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004716,0.0,0.0,0.0,0.0,0.0,0.0,0.004716
4860,7263,Miracle (2004),2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004716,0.0,0.0,0.0,0.0,0.0,0.0,0.004716
1278,1695,Artemisia (1997),1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004716,0.0,0.0,0.0,0.0,0.0,0.0,0.004716


In [26]:
#we sort dfs by movieId

movies = movies.sort_values(by='movie_score')[81:]
movies.sort_values(by='movieId', inplace=True)

In [27]:
ratings.sort_values(by='movieId', inplace=True)

In [28]:
#calculating ratings counts and ratings mean

movie_rating_count = ratings[['movieId','rating']].groupby('movieId').count()
movie_rating_mean = ratings[['movieId','rating']].groupby('movieId').mean()

In [29]:
#we made a new dataframe named movie_analysis

movie_analysis = movie_rating_count.merge(movie_rating_mean, on='movieId', how='inner')
movie_analysis.rename(columns={'rating_x':'rating_count'}, inplace=True)
movie_analysis.rename(columns={'rating_y':'avg_rating'}, inplace=True)
movie_analysis

,rating_count,avg_rating
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429
...,...,...
193581,1,4.000000
193583,1,3.500000
193585,1,3.500000


In [30]:
#we set the threshold (30) now we have 860 movies

movie_analysis = movie_analysis[movie_analysis['rating_count'] > 30]
movie_analysis = movie_analysis.reset_index()
movie_analysis

,movieId,rating_count,avg_rating
0,1,215,3.920930
1,2,110,3.431818
2,3,52,3.259615
3,5,49,3.071429
4,6,102,3.946078
...,...,...,...
855,122904,54,3.833333
856,134130,48,4.000000
857,134853,43,3.813953
858,139385,31,3.903226


In [31]:
#merge movies with movie_analysis
movie_analysis = movie_analysis.merge(movies,on='movieId')
movie_analysis

,movieId,rating_count,avg_rating,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,movie_score
0,1,215,3.920930,Toy Story (1995),1995,0.000000,0.035622,0.151081,0.093469,0.027034,0.000000,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.000000,0.000000,0.377321
1,2,110,3.431818,Jumanji (1995),1995,0.000000,0.035622,0.000000,0.093469,0.000000,0.000000,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.000000,0.000000,0.199206
2,3,52,3.259615,Grumpier Old Men (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.045781,0.000000,0.000000,0.072815
3,5,49,3.071429,Father of the Bride Part II (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.027034
4,6,102,3.946078,Heat (1995),1995,0.028206,0.000000,0.000000,0.000000,0.000000,0.066706,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.029688,0.124600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,122904,54,3.833333,Deadpool (2016),2016,0.028206,0.035622,0.000000,0.000000,0.027034,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.064532,0.000000,0.155394
856,134130,48,4.000000,The Martian (2015),2015,0.000000,0.035622,0.000000,0.000000,0.000000,0.000000,0.0,0.004716,0.000000,0.0,0.0,0.000000,0.064532,0.000000,0.104870
857,134853,43,3.813953,Inside Out (2015),2015,0.000000,0.035622,0.151081,0.093469,0.027034,0.000000,0.0,0.004716,0.070115,0.0,0.0,0.000000,0.000000,0.000000,0.382037
858,139385,31,3.903226,The Revenant (2015),2015,0.000000,0.035622,0.000000,0.000000,0.000000,0.000000,0.0,0.004716,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.040338


In [35]:
movie_analysis['title'].to_csv('./csv/available_movies.csv', index=False, header=False)

In [32]:
#we divide each movies movie_score to genre count that each movie has, so we balanced the repeat count for movies

counter = 0

for index, row in movie_analysis.iterrows():
    for i in range(5,19):
        if row[i] > 0:
            counter +=1
    movie_analysis['movie_score'][index] = row[19] / counter
    counter = 0
    

C:\Users\arsla\AppData\Local\Temp\ipykernel_14380\4179684412.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_analysis['movie_score'][index] = row[19] / counter


In [33]:
#repeat count formula is ((movie_score * total_rating_count / 500 )+ rating_count) for each movie (we divide to 500 because we wanted to balance the repeat count)

movie_analysis = movie_analysis.assign(repeatCount=lambda x: (x.movie_score  * 57358 / 500) + x.rating_count)
movie_analysis = movie_analysis.astype({"repeatCount": int})
movie_analysis

,movieId,rating_count,avg_rating,title,year,Action,Adventure,Animation,Children,Comedy,...,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,movie_score,repeatCount
0,1,215,3.920930,Toy Story (1995),1995,0.000000,0.035622,0.151081,0.093469,0.027034,...,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.000000,0.000000,0.075464,223
1,2,110,3.431818,Jumanji (1995),1995,0.000000,0.035622,0.000000,0.093469,0.000000,...,0.0,0.000000,0.070115,0.0,0.0,0.000000,0.000000,0.000000,0.066402,117
2,3,52,3.259615,Grumpier Old Men (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,...,0.0,0.000000,0.000000,0.0,0.0,0.045781,0.000000,0.000000,0.036407,56
3,5,49,3.071429,Father of the Bride Part II (1995),1995,0.000000,0.000000,0.000000,0.000000,0.027034,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.027034,52
4,6,102,3.946078,Heat (1995),1995,0.028206,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.029688,0.041533,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,122904,54,3.833333,Deadpool (2016),2016,0.028206,0.035622,0.000000,0.000000,0.027034,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.064532,0.000000,0.038849,58
856,134130,48,4.000000,The Martian (2015),2015,0.000000,0.035622,0.000000,0.000000,0.000000,...,0.0,0.004716,0.000000,0.0,0.0,0.000000,0.064532,0.000000,0.034957,52
857,134853,43,3.813953,Inside Out (2015),2015,0.000000,0.035622,0.151081,0.093469,0.027034,...,0.0,0.004716,0.070115,0.0,0.0,0.000000,0.000000,0.000000,0.063673,50
858,139385,31,3.903226,The Revenant (2015),2015,0.000000,0.035622,0.000000,0.000000,0.000000,...,0.0,0.004716,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.020169,33


In [34]:
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movies.set_index('movieId').loc[movie_analysis.set_index('movieId').index].title))
}

In [35]:
with open('./pickles/movie_to_idx.pickle', 'wb') as f:
    # Dump the dictionary into the file using pickle
    pickle.dump(movie_to_idx, f)

In [36]:
movie_to_idx

{'Toy Story (1995)': 0,
 'Jumanji (1995)': 1,
 'Grumpier Old Men (1995)': 2,
 'Father of the Bride Part II (1995)': 3,
 'Heat (1995)': 4,
 'Sabrina (1995)': 5,
 'GoldenEye (1995)': 6,
 'American President, The (1995)': 7,
 'Casino (1995)': 8,
 'Sense and Sensibility (1995)': 9,
 'Ace Ventura: When Nature Calls (1995)': 10,
 'Get Shorty (1995)': 11,
 'Copycat (1995)': 12,
 'Leaving Las Vegas (1995)': 13,
 'City of Lost Children, The (Cité des enfants perdus, La) (1995)': 14,
 'Dangerous Minds (1995)': 15,
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)': 16,
 'Babe (1995)': 17,
 'Dead Man Walking (1995)': 18,
 'Clueless (1995)': 19,
 'Mortal Kombat (1995)': 20,
 'To Die For (1995)': 21,
 'Seven (a.k.a. Se7en) (1995)': 22,
 'Pocahontas (1995)': 23,
 'Usual Suspects, The (1995)': 24,
 'Postman, The (Postino, Il) (1994)': 25,
 'Indian in the Cupboard, The (1995)': 26,
 "Mr. Holland's Opus (1995)": 27,
 'Bio-Dome (1996)': 28,
 'From Dusk Till Dawn (1996)': 29,
 'Broken Arrow (1996)': 30,
 'Happ

In [37]:
#we add repeatCount column to ratings df

ratings = ratings.merge(movie_analysis[['movieId','repeatCount']], on='movieId')
ratings

,userId,movieId,rating,repeatCount
0,1,1,4.0,223
1,517,1,4.0,223
2,213,1,3.5,223
3,514,1,4.0,223
4,214,1,3.0,223
...,...,...,...,...
57353,525,152081,3.0,39
57354,21,152081,4.0,39
57355,114,152081,3.5,39
57356,596,152081,4.0,39


In [38]:
ratings.sort_values(by='userId')

,userId,movieId,rating,repeatCount
0,1,1,4.0,223
28887,1,2018,5.0,47
18635,1,1136,5.0,141
8523,1,362,5.0,40
29070,1,2028,4.0,189
...,...,...,...,...
16408,610,912,3.5,102
39747,610,3977,2.0,75
18349,610,1097,4.5,128
53730,610,60684,3.5,48


In [39]:
user_rating_count = ratings[['userId','rating']].groupby('userId').count()
user_rating_mean = ratings[['userId','rating']].groupby('userId').mean()

In [40]:
#created user_analysis dataframe

user_analysis = user_rating_count.merge(user_rating_mean, on='userId', how='inner')
user_analysis.rename(columns={'rating_x':'rating_count'}, inplace=True)
user_analysis.rename(columns={'rating_y':'avg_rating'}, inplace=True)
user_analysis

,rating_count,avg_rating
userId,,
1,157,4.401274
2,19,3.868421
3,19,1.736842
4,116,3.431034
5,37,3.540541
...,...,...
606,363,3.663912
607,126,3.730159
608,479,3.339248


In [41]:
#then we add these features to ratings dataframe

ratings = ratings.merge(user_analysis, on='userId', how='left')
ratings

,userId,movieId,rating,repeatCount,rating_count,avg_rating
0,1,1,4.0,223,157,4.401274
1,517,1,4.0,223,220,2.550000
2,213,1,3.5,223,47,4.021277
3,514,1,4.0,223,196,3.767857
4,214,1,3.0,223,17,2.823529
...,...,...,...,...,...,...
57353,525,152081,3.0,39,312,3.639423
57354,21,152081,4.0,39,226,3.398230
57355,114,152081,3.5,39,27,3.481481
57356,596,152081,4.0,39,249,3.483936


In [42]:
#creating 'umr_dict' (userId, movieId, rating dictionary) and export so we will use this when we are predicting

inner_dict_name = 'movies'
def create_inner_dict(group):
    return {inner_dict_name: dict(zip(group['movieId'], group['rating']))}

umr_dict = ratings[['userId','movieId','rating']].groupby('userId').apply(create_inner_dict).to_dict()

with open('./pickles/umr_dict.pickle', 'wb') as f:
    # Dump the dictionary into the file using pickle
    pickle.dump(umr_dict, f)

In [43]:
#augmenting data with repeating

ratings = ratings.loc[ratings.index.repeat(ratings['repeatCount'])]
ratings

,userId,movieId,rating,repeatCount,rating_count,avg_rating
0,1,1,4.0,223,157,4.401274
0,1,1,4.0,223,157,4.401274
0,1,1,4.0,223,157,4.401274
0,1,1,4.0,223,157,4.401274
0,1,1,4.0,223,157,4.401274
...,...,...,...,...,...,...
57357,401,152081,4.0,39,48,3.458333
57357,401,152081,4.0,39,48,3.458333
57357,401,152081,4.0,39,48,3.458333
57357,401,152081,4.0,39,48,3.458333


In [44]:
item_vecs = movie_analysis[['movieId','year','avg_rating']]
movie_analysis = movie_analysis[['movieId','avg_rating']]
item_vecs

,movieId,year,avg_rating
0,1,1995,3.920930
1,2,1995,3.431818
2,3,1995,3.259615
3,5,1995,3.071429
4,6,1995,3.946078
...,...,...,...
855,122904,2016,3.833333
856,134130,2015,4.000000
857,134853,2015,3.813953
858,139385,2015,3.903226


In [45]:
item_vecs = item_vecs.merge(movies_w_genres, on='movieId')
item_vecs = item_vecs.drop('genres', axis=1)

movie_analysis = movie_analysis.merge(movies_w_genres, on='movieId')
movie_analysis = movie_analysis.drop('genres', axis=1)

In [46]:
item_vecs.drop('title',axis=1,inplace=True)
item_vecs.to_csv('item_vecs.csv',index=False, header=False)

In [47]:
movie_analysis

,movieId,avg_rating,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,3.920930,Toy Story (1995),0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,3.431818,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,3.259615,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,5,3.071429,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6,3.946078,Heat (1995),1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,122904,3.833333,Deadpool (2016),1,1,0,0,1,0,0,0,0,0,0,0,1,0
856,134130,4.000000,The Martian (2015),0,1,0,0,0,0,0,1,0,0,0,0,1,0
857,134853,3.813953,Inside Out (2015),0,1,1,1,1,0,0,1,1,0,0,0,0,0
858,139385,3.903226,The Revenant (2015),0,1,0,0,0,0,0,1,0,0,0,0,0,0


In [48]:
genreCalcDF = pd.DataFrame(movie_analysis)
genreCalcDF.drop(columns=['avg_rating','title'], axis=1, inplace=True)
genreCalcDF

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,122904,1,1,0,0,1,0,0,0,0,0,0,0,1,0
856,134130,0,1,0,0,0,0,0,1,0,0,0,0,1,0
857,134853,0,1,1,1,1,0,0,1,1,0,0,0,0,0
858,139385,0,1,0,0,0,0,0,1,0,0,0,0,0,0


In [49]:
genreCalcDF = pd.merge(ratings, genreCalcDF, on = 'movieId')
genreCalcDF

,userId,movieId,rating,repeatCount,rating_count,avg_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1,4.0,223,157,4.401274,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,1,1,4.0,223,157,4.401274,0,1,1,1,1,0,0,0,1,0,0,0,0,0
2,1,1,4.0,223,157,4.401274,0,1,1,1,1,0,0,0,1,0,0,0,0,0
3,1,1,4.0,223,157,4.401274,0,1,1,1,1,0,0,0,1,0,0,0,0,0
4,1,1,4.0,223,157,4.401274,0,1,1,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5630126,401,152081,4.0,39,48,3.458333,1,1,1,1,1,0,0,0,0,0,0,0,0,0
5630127,401,152081,4.0,39,48,3.458333,1,1,1,1,1,0,0,0,0,0,0,0,0,0
5630128,401,152081,4.0,39,48,3.458333,1,1,1,1,1,0,0,0,0,0,0,0,0,0
5630129,401,152081,4.0,39,48,3.458333,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [50]:
columns = genreCalcDF.columns
columns = columns.drop(['userId','rating','movieId','repeatCount','rating_count','avg_rating'])
columns

Index(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller'],
      dtype='object')

In [51]:
for i in columns:
    genreCalcDF[i] = genreCalcDF[i] * genreCalcDF['rating']
genreCalcDF.sort_values('userId', inplace=True)
genreCalcDF

,userId,movieId,rating,repeatCount,rating_count,avg_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1,4.0,223,157,4.401274,0.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3522611,1,2353,5.0,70,157,4.401274,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3522610,1,2353,5.0,70,157,4.401274,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3522609,1,2353,5.0,70,157,4.401274,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3522608,1,2353,5.0,70,157,4.401274,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697665,610,2628,3.0,144,405,4.046914,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3697664,610,2628,3.0,144,405,4.046914,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3697663,610,2628,3.0,144,405,4.046914,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3697675,610,2628,3.0,144,405,4.046914,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [52]:
genreCalcDF.replace(0, np.nan, inplace=True)

grouped = genreCalcDF[['userId','rating','Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller']].groupby(['userId']).mean()
grouped.replace(np.nan, 0, inplace=True)
grouped = grouped.reset_index()
grouped

,userId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.000000,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
1,2,3.831426,4.012935,4.036304,0.000000,0.000000,3.988166,3.760476,0.000000,3.866120,0.000000,3.000000,4.000000,4.500000,3.896057,3.938944
2,3,1.491449,3.914336,3.738596,0.500000,0.500000,0.500000,0.000000,0.000000,0.500000,2.506623,4.000000,0.000000,0.500000,3.732794,4.643357
3,4,3.404464,3.205338,3.837613,3.849810,4.023121,3.371575,3.459840,0.000000,3.212456,4.121255,4.506173,2.738629,3.263917,2.718346,2.866915
4,5,3.575807,3.100407,3.294612,4.147577,4.037986,3.505860,3.643135,0.000000,3.760920,4.224909,3.000000,4.000000,2.954600,2.613941,3.650548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,606,3.747637,3.561907,3.618071,3.683623,3.608268,3.564808,3.951020,4.000000,3.917556,3.698100,3.770227,3.804289,3.792151,3.647882,3.775536
605,607,3.836736,3.782987,3.502354,3.357285,3.650565,3.347939,3.864714,0.000000,4.081291,3.484096,4.152187,4.807554,3.644503,3.484638,4.033825
606,608,3.426483,3.526191,3.363805,3.162697,2.873238,3.120970,3.805606,3.353723,3.578827,3.214980,3.731632,3.926115,3.071267,3.518950,3.714096
607,609,3.383508,3.082217,3.209028,3.000000,3.000000,3.581670,3.609073,0.000000,3.562641,3.000000,4.000000,0.000000,3.702760,3.000000,3.361473


In [53]:
genreCalcDF = genreCalcDF.iloc[:,:5]
genreCalcDF

,userId,movieId,rating,repeatCount,rating_count
0,1,1,4.0,223,157
3522611,1,2353,5.0,70,157
3522610,1,2353,5.0,70,157
3522609,1,2353,5.0,70,157
3522608,1,2353,5.0,70,157
...,...,...,...,...,...
3697665,610,2628,3.0,144,405
3697664,610,2628,3.0,144,405
3697663,610,2628,3.0,144,405
3697675,610,2628,3.0,144,405


In [54]:
user_df = genreCalcDF.merge(grouped, on='userId', how='left')
user_df

,userId,movieId,rating_x,repeatCount,rating_count,rating_y,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1,4.0,223,157,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.0,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
1,1,2353,5.0,70,157,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.0,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
2,1,2353,5.0,70,157,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.0,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
3,1,2353,5.0,70,157,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.0,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
4,1,2353,5.0,70,157,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.0,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5630126,610,2628,3.0,144,405,4.190215,4.248085,4.225753,4.152609,4.108337,4.054981,4.265625,4.0,4.189497,4.170596,4.261021,4.350904,3.980188,4.289857,4.327551
5630127,610,2628,3.0,144,405,4.190215,4.248085,4.225753,4.152609,4.108337,4.054981,4.265625,4.0,4.189497,4.170596,4.261021,4.350904,3.980188,4.289857,4.327551
5630128,610,2628,3.0,144,405,4.190215,4.248085,4.225753,4.152609,4.108337,4.054981,4.265625,4.0,4.189497,4.170596,4.261021,4.350904,3.980188,4.289857,4.327551
5630129,610,2628,3.0,144,405,4.190215,4.248085,4.225753,4.152609,4.108337,4.054981,4.265625,4.0,4.189497,4.170596,4.261021,4.350904,3.980188,4.289857,4.327551


In [55]:
movie_analysis = movie_analysis.merge(movies[['movieId','year']], on='movieId', how='left')
movie_analysis = movie_analysis.reindex(columns=['movieId','year','avg_rating','Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller'])
movie_analysis

,movieId,year,avg_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,1995,3.431818,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,1995,3.259615,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,5,1995,3.071429,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6,1995,3.946078,1,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,122904,2016,3.833333,1,1,0,0,1,0,0,0,0,0,0,0,1,0
856,134130,2015,4.000000,0,1,0,0,0,0,0,1,0,0,0,0,1,0
857,134853,2015,3.813953,0,1,1,1,1,0,0,1,1,0,0,0,0,0
858,139385,2015,3.903226,0,1,0,0,0,0,0,1,0,0,0,0,0,0


In [56]:
item_df = movie_analysis.merge(genreCalcDF['movieId'], on='movieId', how='left')
item_df.sort_values(by='movieId', inplace=True)
item_df

,movieId,year,avg_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
31958,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
31959,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
31960,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
31961,1,1995,3.920930,0,1,1,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629295,152081,2016,3.890625,1,1,1,1,1,0,0,0,0,0,0,0,0,0
5629294,152081,2016,3.890625,1,1,1,1,1,0,0,0,0,0,0,0,0,0
5629293,152081,2016,3.890625,1,1,1,1,1,0,0,0,0,0,0,0,0,0
5629300,152081,2016,3.890625,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [57]:
user_df.sort_values(by='movieId',inplace=True)
y_df = user_df['rating_x']
y_df
user_df.drop(['movieId','rating_x','repeatCount'],axis=1,inplace=True)
user_df.rename(columns={'rating_y': 'rating_average'}, inplace=True)
user_df

,userId,rating_count,rating_average,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,157,4.386339,4.314322,4.406360,4.642857,4.712817,4.248431,4.357483,0.000000,4.399524,4.447579,3.279826,4.491241,4.252231,4.332343,4.221207
4676867,517,220,2.662250,2.153441,2.719302,3.203488,3.376324,2.742244,2.309676,0.000000,2.723879,3.077920,1.905612,1.723588,2.843044,2.597649,2.078313
4676866,517,220,2.662250,2.153441,2.719302,3.203488,3.376324,2.742244,2.309676,0.000000,2.723879,3.077920,1.905612,1.723588,2.843044,2.597649,2.078313
4676865,517,220,2.662250,2.153441,2.719302,3.203488,3.376324,2.742244,2.309676,0.000000,2.723879,3.077920,1.905612,1.723588,2.843044,2.597649,2.078313
4676864,517,220,2.662250,2.153441,2.719302,3.203488,3.376324,2.742244,2.309676,0.000000,2.723879,3.077920,1.905612,1.723588,2.843044,2.597649,2.078313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632556,401,48,3.393764,3.226357,3.427546,3.513661,3.485444,3.445603,3.500000,0.000000,3.464067,3.382743,0.000000,0.000000,3.460123,3.043278,4.000000
3632555,401,48,3.393764,3.226357,3.427546,3.513661,3.485444,3.445603,3.500000,0.000000,3.464067,3.382743,0.000000,0.000000,3.460123,3.043278,4.000000
3632554,401,48,3.393764,3.226357,3.427546,3.513661,3.485444,3.445603,3.500000,0.000000,3.464067,3.382743,0.000000,0.000000,3.460123,3.043278,4.000000
2270690,252,18,4.425980,3.775763,4.513595,4.651385,4.691099,4.593142,3.500000,0.000000,4.498829,4.588068,0.000000,3.500000,0.000000,3.697468,3.729167


In [58]:
y_df

0          4.0
4676867    4.0
4676866    4.0
4676865    4.0
4676864    4.0
          ... 
3632556    4.0
3632555    4.0
3632554    4.0
2270690    4.0
4764973    3.0
Name: rating_x, Length: 5630131, dtype: float64

CSV!!!!!!11

In [59]:
item_df.to_csv('csv/x_item.csv',index=False, header=False)
#user_df.to_csv('csv/x_user.csv',index=False, header=False)
#y_df.to_csv('csv/y.csv',index=False, header=False)

